In [451]:
#import Ipynb_importer
#import DbOpModule
import pymysql
import tushare as ts
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import talib as ta
import sys
import datetime

In [452]:
class MysqlOp():
    
    db = None
    cursor = None
    
    
    def __init__(self):
        db = None
        cursor = None
    
    def connect_To_Mysql(self, user_ = 'root', password_ ='Aa123456' , db_="dbtest"):
            try:
                self.db = pymysql.connect(host='localhost',
                         user = user_,
                         password = password_ ,
                         database = db_,
                         charset = 'utf8')
                print("数据库连接成功！")
            except pymysql.Error as e:
                print("连接数据库错误："+str(e))
            
            self.cursor = self.db.cursor()
            
            
            
    def create_stock_daily_table(self,table_name):
            try:
                VALUE = table_name
                sql = """CREATE TABLE {:s} (id int not null auto_increment PRIMARY  KEY, ts_code CHAR(9) NOT NULL,trade_date CHAR(8), open CHAR(10),
                   high CHAR(10), low CHAR(10),close CHAR(10), pre_close CHAR(10), pri_chg CHAR(12), pct_chg CHAR(10), vol CHAR(16), amount CHAR(16) )""".format(VALUE)
                self.cursor.execute(sql)
                self.db.commit()
                print ("Create Table: " + VALUE)
            except  Exception as e:
                #print("Create Table False!!!" + "fdftable_name")
                print("Create Table False!! Error: " +str(e) +"  ---------------fdsafsfsaf---2222-------")

    def create_stock_daily_table_2(self, table_name):
            try:
                VALUE = table_name
                sql = """CREATE TABLE {:s} (ts_code CHAR(12) NOT NULL,
                                            trade_date CHAR(12), 
                                            open double(9,4),
                                            high double(9,4), 
                                            low double(9,4),
                                            close double(9,4), 
                                            pre_close double(9,4), 
                                            pri_chg double(9,4), 
                                            pct_chg double(9,4), 
                                            vol double(12,4), 
                                            amount double(12,4),
                                            PRIMARY KEY(ts_code,trade_date) 
                                            )""".format(VALUE)
                self.cursor.execute(sql)
                self.db.commit()
                print ("Create Table: " + VALUE)
            except  Exception as e:
                #print("Create Table False!!!" + "fdftable_name")
                print("Create Table False!! Error: " +str(e) +"  ---------------fdsafsfsaf---2222-------")
                   

    def del_database_table(self,table_name ):
        Value = table_name
        try:
            sql='DROP TABLE IF EXISTS {:s}'.format(Value)    
            self.cursor.execute(sql)
            self.db.commit()
            print("Drop Table successed !!")
        except  Exception as e:
            print("Drop Table False!! Error: " +str(e) )
              

    def Insert_stock_daily(self, table_name, df):
        #sql_insert = "INSERT INTO Stock_basic(stock_code,date,open,high,close,low,volume,chg,chg_percent,ma5,ma10,\
        #        ma20,vma5,vma10,vma20,turnover) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        Value = table_name
        sql_insert = """INSERT INTO {:s}(ts_code, trade_date, open, high, low, close, pre_close, pri_chg, pct_chg, vol,
                      amount) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""".format(Value)
        for indexs in df.index:
            try:
                self.cursor.execute(sql_insert,(str(df.loc[indexs,'ts_code']),str(df.loc[indexs,'trade_date']),str(df.loc[indexs,'open']),str(df.loc[indexs,'high']),\
                                    str(df.loc[indexs,'low']),str(df.loc[indexs,'close']),str(df.loc[indexs,'pre_close']),str(round(df.loc[indexs,'change'],4)),\
                                    str(df.loc[indexs,'pct_chg']),str(df.loc[indexs,'vol']),str(df.loc[indexs,'amount']) ))
                self.db.commit()
                
                print("写入成功一条股票日线数据！" + str(indexs) )
            except Exception as e:
                print("写入日线数据失败！！！ Error : "+ str(e) + "    " + str(indexs) +"   " + str(df.loc[indexs,'pre_close']))        

    def Insert_stock_daily_2(self, table_name, df):
        #sql_insert = "INSERT INTO Stock_basic(stock_code,date,open,high,close,low,volume,chg,chg_percent,ma5,ma10,\
        #        ma20,vma5,vma10,vma20,turnover) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        Value = table_name
        sql_insert = """INSERT INTO {:s}(ts_code, trade_date, open, high, low, close, pre_close, pri_chg, pct_chg, vol,
                      amount) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""".format(Value)
        
        for i in df.index:
            try:
                self.cursor.execute(sql_insert, (df.loc[i,'ts_code'],\
                                                df.loc[i,'trade_date'],\
                                                df.loc[i,'open'],\
                                                df.loc[i,'high'],\
                                                df.loc[i,'low'],\
                                                df.loc[i,'close'],\
                                                df.loc[i,'pre_close'],\
                                                df.loc[i,'change'],\
                                                df.loc[i,'pct_chg'],\
                                                df.loc[i,'vol'],\
                                                df.loc[i,'amount'] ))
                
                if(((i % 1000) == 0) and  (i !=0)):  # 每 1000 次 insert ,调用一次 commit ,    
                    self.db.commit()
                    print("写入成功1000条股票日线数据！ 当前index: " + str(i) )
            except Exception as e:
                print("写入日线数据失败！！！ Error : "+ str(e) + "    " + str(i) +"   " + str(df.loc[indexs,'pre_close']))        

        self.db.commit()   # 每 1000 次 insert ,调用一次 commit ,  最后循环完成后，也做一次 commit
        print("写入成功一次for 循环的内容 " + df.loc[i,'ts_code'] + "  当前index: " + str(df.index) )
            

    def Insert_many_stock_daily(self, table_name, df):
        #sql_insert = "INSERT INTO Stock_basic(stock_code,date,open,high,close,low,volume,chg,chg_percent,ma5,ma10,\
        #        ma20,vma5,vma10,vma20,turnover) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        Value = table_name
        sql_insert = "INSERT INTO {:s}(ts_code, trade_date, open, high, low, close, pre_close, pri_chg, pct_chg, vol, amount) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)".format(Value)
        
        data= []
        for i in df.index:
           data.append([df.loc[i,'ts_code'],
                        df.loc[i,'trade_date'],
                        round(df.loc[i,'open'],4),
                        round(df.loc[i,'high'],4),
                        round(df.loc[i,'low'],4),
                        round(df.loc[i,'close'],4),
                        round(df.loc[i,'pre_close'],4),
                        round(df.loc[i,'change'],4),
                        round(df.loc[i,'pct_chg'],4),
                        round(df.loc[i,'vol'],4),
                        round(df.loc[i,'amount'],4)] )


        try:
            self.cursor.executemany(sql_insert, data )
            self.db.commit()
            print("成功写入股票日线数据！ code: " + str(df.loc[0,'ts_code']) +"   " + str(df.index) + "  datasize:"+ str(sys.getsizeof(data)))
        except Exception as e:
            print("写入日线数据失败！！！ Error : "+ str(e) + "    code: "  + str(df.loc[0,'ts_code']))        

            


 #   def get_stock_daily_form_db(self, code, star_date = '20200101', end_date = '20220728'):
    def get_stock_daily_form_db_(self,table_name):
        Value = table_name
        sql =  'SELECT * FROM {:s}'.format(Value)
        #sql =  'SELECT * FROM stock_daily'
        self.cursor.execute(sql)
        result = self.cursor.fetchall()
        return result


    def del_all_data(self,table_name):
        try:
            Value = table_name
            sql = " DELETE FROM {:s} ".format(Value)
            self.cursor.execute(sql)
            self.db.commit()
            print("Del table data successed!")
        except Exception as e:
            print("Del table data False!!False!!!! Error : "+ str(e) )

    def get_daily_data_from_db(self,table_name,code):
        Value_01 = table_name
        Value_02 = code
        sql =  'SELECT * FROM {:s} WHERE ts_code = %s '.format( Value_01)
        self.cursor.execute(sql,Value_02)
        result = self.cursor.fetchall()

        df = pd.DataFrame(result)
       # df.columns = ['ts_code','trade_date','open','high','low','close','pre_close','change','pct_chg','vol','amount']
        #df.loc[:,'ts_code':'amount']
        return df

    def get_daily_data_from_db_2(self,table_name):
        Value_01 = table_name
        sql =  'SELECT * FROM test_info WHERE trade_date > 20220629 '
        self.cursor.execute(sql)
        result = self.cursor.fetchall()

        df = pd.DataFrame(result)
        if( len(df) >0):
            df.columns = ['ts_code','trade_date','open','high','low','close','pre_close','change','pct_chg','vol','amount']
            
        return df

    # 从数据库读取股票日线数据， 每次读取一支股票， 
    # 股票代码是必要的参数
    # 开始和结束日期是可选参数，如果不传入，读取全部历史数据
    # 如果传入，读取指定日期数据
    def get_stock_daily_from_db(self,table_name,code, star_date = '19910101', end_date = datetime.datetime.now().strftime('%Y%m%d')):
        Value = (code, star_date, end_date)
        sql =  'SELECT * FROM {:s} WHERE ts_code = %s and trade_date > %s and trade_date < %s '.format(table_name)
        self.cursor.execute(sql, Value)
        result = self.cursor.fetchall()

        df = pd.DataFrame(result, columns = ['ts_code','trade_date','open','high','low','close','pre_close','change','pct_chg','vol','amount'])     
        return df
        
    def get_one_day_stock_info(self, table_name, date):
        #Value_01 = table_name
        sql =  'SELECT * FROM {:s} WHERE trade_date = {:s} '.format(table_name,date)
        self.cursor.execute(sql)
        result = self.cursor.fetchall()

        df = pd.DataFrame(result,columns = ['ts_code','trade_date','open','high','low','close','pre_close','change','pct_chg','vol','amount'])
   
        return df

    def get_all_stock_code(self,table_name):
        sql =  'SELECT DISTINCT ts_code FROM {:s} '.format(table_name)
        self.cursor.execute(sql)
        result = self.cursor.fetchall()

        return result


    def close_db(self):
        self.cursor.close()
        self.db.close()

    def foo(self):
        print("Test!!!!!!!!!!!!")


    

In [453]:
obj_db = MysqlOp()
obj_db.connect_To_Mysql(db_ = 'stockdb')

数据库连接成功！


In [454]:
#dataframe = pd.read_csv("002920.csv")
#dataframe.loc[0:10]

In [455]:
table_name = "stock_daily"
#obj_db.del_database_table(table_name)

In [456]:
#obj_db.create_stock_daily_table("stock_daily")
#obj_db.create_stock_daily_table_2(table_name)

In [457]:
sb=pd.read_csv('stockInfo\StockBasic.csv')
sb

,ts_code,symbol,name,area,industry,list_date
0,000001.SZ,1,平安银行,深圳,银行,19910403
1,000002.SZ,2,万科A,深圳,全国地产,19910129
2,000004.SZ,4,ST国华,深圳,软件服务,19910114
3,000005.SZ,5,ST星源,深圳,环境保护,19901210
4,000006.SZ,6,深振业A,深圳,区域地产,19920427
...,...,...,...,...,...,...
4839,871981.BJ,871981,晶赛科技,NaN,NaN,20211115
4840,872925.BJ,872925,锦好医疗,NaN,NaN,20211025
4841,873169.BJ,873169,七丰精工,NaN,NaN,20220415
4842,873223.BJ,873223,荣亿精密,NaN,NaN,20220609


In [458]:
sb.index

RangeIndex(start=0, stop=4844, step=1)

In [459]:
def store_stock_daily_to_mySQL(ref_df):
    sb = ref_df
    for i in sb.index:

        #print(" Processing : "  + sb.loc[i,'ts_code'] +"  " + str(sb.loc[i,'list_date']))
        if( int(sb.loc[i,'list_date'])  < 20021231):  # 如果上市日期早于 20081231， 刚分 2 次取数据
            print(" Processing 2 step: "  +  "  sb.index:" + str(i) + "  " + sb.loc[i,'ts_code'] +"  " + str(sb.loc[i,'list_date']))
            df  = ts.pro_bar(ts_code=sb.loc[i,'ts_code'], adj='qfq', start_date= str(sb.loc[i,'list_date']), end_date='20021231')
            df2 = ts.pro_bar(ts_code=sb.loc[i,'ts_code'], adj='qfq', start_date= '20030101' , end_date='20220803')
            df3 = pd.concat([df2,df],ignore_index=True)

            # 如果返回数据为 0， 跳过本次
            if(  df3.empty == True):
                continue

            if( df3.isnull().values.any() == True ):
                df3.fillna(method="ffill", inplace=True)

            obj_db.Insert_many_stock_daily(table_name =table_name, df =df3 )   

        else:
            print(" Processing 1 step: "  +  "  sb.index:" + str(i) + "  " + sb.loc[i,'ts_code'] +"  " + str(sb.loc[i,'list_date']))
            df = ts.pro_bar(ts_code=sb.loc[i,'ts_code'], adj='qfq', start_date= str(sb.loc[i,'list_date']), end_date='20220802')

            # 如果返回数据为 0， 跳过本次
            if( df is None ):
                continue

            if( df.isnull().values.any() == True ):
                df.fillna(method="ffill", inplace=True)
                
            obj_db.Insert_many_stock_daily(table_name =table_name, df =df )
    

In [460]:
pro = ts.pro_api('af5f423bb57f376e71c2903058362f8e3e053fe30f486463f9bdac8e')

df = ts.pro_bar(ts_code=sb.loc[0,'ts_code'], adj='qfq', start_date= str(sb.loc[0,'list_date']), end_date='20041231')
            #obj_db.Insert_stock_daily_3(table_name =table_name, df =df )
df2 = ts.pro_bar(ts_code=sb.loc[0,'ts_code'], adj='qfq', start_date= '20050101' , end_date='20220803')


df3 = pd.concat([df2,df],ignore_index=True)
df3

In [461]:
#store_stock_daily_to_mySQL()

df = ts.pro_bar(ts_code=sb.loc[3120    ,'ts_code'], adj='qfq', start_date= str(sb.loc[3120    ,'list_date']), end_date='20021231')
            #obj_db.Insert_stock_daily_3(table_name =table_name, df =df )
df2 = ts.pro_bar(ts_code=sb.loc[3120    ,'ts_code'], adj='qfq', start_date= '20030101' , end_date='20220803')

df3 = pd.concat([df2,df],ignore_index=True)
  
#df3.to_csv("600600.csv",index = False)
df3[6864:6866]



df3[6844:6866]

if( df3.isnull().values.any() == True ):
    df3.fillna(method="ffill", inplace=True)

df3.isnull().values.any()

In [462]:
#obj_db.del_all_data(table_name)

In [463]:
#dataframe = pd.read_csv('002920.csv')
#dataframe02 = pd.read_csv('000001.csv')

In [464]:
#obj_db.Insert_stock_daily(table_name =table_name, df =dataframe.loc[50:100]  )
#obj_db.Insert_stock_daily_2(table_name =table_name, df =dataframe.loc[0:10]  )
#obj_db.Insert_many_stock_daily(table_name =table_name, df =dataframe02.loc[0:10]  )

In [465]:
#val_ret  = obj_db.get_stock_daily_form_db(table_name = table_name)
#len(val_ret)

In [466]:
#obj_db.del_all_data(table_name =table_name)

In [467]:
val_ret = obj_db.get_daily_data_from_db(table_name =table_name, code ="000001.SZ")
val_ret
type(val_ret)

pandas.core.frame.DataFrame

In [468]:
#da = obj_db.get_one_day_stock_info(table_name =table_name,date ="20220726")

#sys.getsizeof(val_ret)

In [469]:
da

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,20220726,12.75,12.91,12.71,12.91,12.82,0.09,0.7020,1024048.81,1314310.442
1,000002.SZ,20220726,17.50,18.19,17.23,18.05,17.50,0.55,3.1429,1157357.50,2067176.291
2,000004.SZ,20220726,9.30,9.58,9.30,9.48,9.30,0.18,1.9355,38874.39,36790.695
3,000005.SZ,20220726,1.79,1.88,1.78,1.85,1.80,0.05,2.7778,161478.40,29729.229
4,000006.SZ,20220726,4.18,4.30,4.16,4.29,4.18,0.11,2.6316,155196.82,65952.030
...,...,...,...,...,...,...,...,...,...,...,...
4827,871970.BJ,20220726,9.27,9.42,9.27,9.41,9.27,0.14,1.5102,1250.12,1169.731
4828,871981.BJ,20220726,18.17,18.84,18.10,18.70,17.98,0.72,4.0044,2379.31,4395.094
4829,872925.BJ,20220726,16.08,16.17,15.96,16.13,16.03,0.10,0.6238,183.16,294.104
4830,873169.BJ,20220726,6.26,6.34,6.20,6.34,6.24,0.10,1.6026,4465.24,2799.661


In [470]:
#obj_db.close_db()

In [471]:
temp2 = da.loc[:,"ts_code"].values
temp2 = list(temp2)
len(temp2)

4832

In [472]:
temp1 = sb.loc[:,"ts_code"].values
temp1 = list(temp1)
len(temp1)

4844

In [473]:
ss =list(set(temp1) - set(temp2))
ss

['002883.SZ',
 '603603.SH',
 '688253.SH',
 '834415.BJ',
 '301195.SZ',
 '600647.SH',
 '603201.SH',
 '688130.SH',
 '301269.SZ',
 '001258.SZ',
 '300070.SZ',
 '300013.SZ']

In [474]:
def get_diff_2df(df1,df2):
    temp1 = df1.loc[:,"ts_code"].values
    temp1 = list(temp1)
    temp2 = df2.loc[:,"ts_code"].values
    temp2 = list(temp2)

    ss =list(set(temp1) - set(temp2))

    if( len(ss) >0):
        diff = df1[df1.ts_code == ss[0]]
        for i in range(1,len(ss)):
            diff = pd.concat([diff, df1[df1.ts_code == ss[i]]], ignore_index=True)
        return diff
    return None

In [475]:
bb = sb[sb.ts_code == ss[0]]

for i in range(1,len(ss)):
    bb = pd.concat([bb, sb[sb.ts_code == ss[i]]], ignore_index=True)

bb


,ts_code,symbol,name,area,industry,list_date
0,002883.SZ,2883,中设股份,江苏,建筑工程,20170620
1,603603.SH,603603,*ST博天,北京,环境保护,20170217
2,688253.SH,688253,C英诺特,北京,医疗保健,20220728
3,834415.BJ,834415,恒拓开源,NaN,NaN,20200727
4,301195.SZ,301195,北路智控,江苏,软件服务,20220801
5,600647.SH,600647,同达创业,上海,综合类,19930504
6,603201.SH,603201,N常润,江苏,汽车配件,20220729
7,688130.SH,688130,N晶华微,浙江,半导体,20220729
8,301269.SZ,301269,N华大九,北京,半导体,20220729
9,001258.SZ,1258,立新能源,新疆,新型电力,20220727


In [476]:
#store_stock_daily_to_mySQL(bb)

In [477]:
data = obj_db.get_stock_daily_from_db(table_name=table_name,code ="002920.SZ")
data

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,002920.SZ,20171226,23.8559,28.6271,23.8559,28.6271,19.8832,8.7439,43.9763,597.15,1752.288
1,002920.SZ,20171227,31.4898,31.4898,31.4898,31.4898,28.6271,2.8627,10.0000,253.00,818.202
2,002920.SZ,20171228,34.6349,34.6349,34.6349,34.6349,31.4898,3.1451,9.9877,336.70,1197.641
3,002920.SZ,20171229,38.1013,38.1013,38.1013,38.1013,34.6349,3.4664,10.0084,1504.58,5887.421
4,002920.SZ,20180102,41.9085,41.9085,41.9085,41.9085,38.1013,3.8072,9.9923,27117.87,116715.312
...,...,...,...,...,...,...,...,...,...,...,...
1112,002920.SZ,20220727,176.8400,185.6000,175.0900,183.0200,176.8400,6.1800,3.4947,27631.72,502104.961
1113,002920.SZ,20220728,184.0000,189.6800,180.1800,186.1700,183.0200,3.1500,1.7211,27824.68,517725.873
1114,002920.SZ,20220729,186.0000,203.9900,184.0100,195.0000,186.1700,8.8300,4.7430,39873.96,776912.387
1115,002920.SZ,20220801,193.0000,197.5000,189.0000,196.9000,195.0000,1.9000,0.9744,37507.27,724683.586


In [478]:
data2 = obj_db.get_stock_daily_from_db(table_name=table_name,code ="002920.SZ", star_date="20200101", end_date="20220727")
data2

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,002920.SZ,20200102,30.4011,30.6882,29.9257,30.4704,30.0347,0.4357,1.4507,27953.81,85890.086
1,002920.SZ,20200103,30.4506,31.0348,30.0743,30.9853,30.4704,0.5149,1.6898,30060.09,92500.697
2,002920.SZ,20200106,30.6981,32.9460,30.5991,31.6488,30.9853,0.6635,2.1413,57880.38,186984.730
3,002920.SZ,20200107,31.5102,33.3322,31.0051,32.7777,31.6488,1.1289,3.5670,62478.93,204415.697
4,002920.SZ,20200108,32.1637,33.8472,32.1637,33.5204,32.7777,0.7427,2.2659,62866.20,209906.770
...,...,...,...,...,...,...,...,...,...,...,...
616,002920.SZ,20220720,176.9200,177.6400,168.9400,176.0000,179.2000,-3.2000,-1.7857,54985.90,952927.823
617,002920.SZ,20220721,176.0900,188.4900,172.5000,178.8000,176.0000,2.8000,1.5909,50437.68,916840.192
618,002920.SZ,20220722,179.1700,180.8200,174.0000,179.7200,178.8000,0.9200,0.5145,30540.30,541713.331
619,002920.SZ,20220725,179.7000,179.7000,169.3000,173.8800,179.7200,-5.8400,-3.2495,37140.86,641765.859


In [479]:
now =datetime.datetime.now().strftime('%Y%m%d')

now

'20220805'

In [480]:
sb2 = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
sb2

,ts_code,symbol,name,area,industry,list_date
0,000996.SZ,000996,中国中期,北京,汽车服务,20000718
1,002008.SZ,002008,大族激光,深圳,电器仪表,20040625
2,002014.SZ,002014,永新股份,安徽,塑料,20040708
3,002026.SZ,002026,山东威达,山东,机械基件,20040727
4,002033.SZ,002033,丽江股份,云南,旅游景点,20040825
...,...,...,...,...,...,...
4848,837092.BJ,837092,汉鑫科技,None,None,20211115
4849,839680.BJ,839680,广道高新,None,None,20211115
4850,870436.BJ,870436,大地电气,None,None,20211115
4851,871981.BJ,871981,晶赛科技,None,None,20211115


In [481]:
diff = get_diff_2df(sb2,sb)
diff

,ts_code,symbol,name,area,industry,list_date
0,301333.SZ,301333,C诺思格,北京,医疗保健,20220802
1,688371.SH,688371,C菲沃泰,None,None,20220802
2,301095.SZ,301095,广立微,浙江,半导体,20220805
3,688380.SH,688380,中微半导,深圳,半导体,20220805
4,688373.SH,688373,盟科药业,上海,化学制药,20220805
5,301308.SZ,301308,江波龙,深圳,IT设备,20220805
6,301278.SZ,301278,N快可,江苏,电气设备,20220804
7,001236.SZ,001236,弘业期货,江苏,多元金融,20220805
8,835985.BJ,835985,海泰新能,None,None,20220808


In [482]:
#store_stock_daily_to_mySQL(diff)

In [491]:
sc = obj_db.get_all_stock_code(table_name= table_name)
scl=[]
for i in range(0,len(sc)):
    scl.append(sc[i][0])

scl

4846

In [494]:
temp1 = sb2.loc[:,"ts_code"].values
temp1 = list(temp1)

temp2 = list(scl)

ss =list(set(temp1) - set(temp2))
len(ss)

7

In [496]:
if( len(ss) >0):
    diff = sb2[sb2.ts_code == ss[0]]
    for i in range(1,len(ss)):
        diff = pd.concat([diff, sb2[sb2.ts_code == ss[i]]], ignore_index=True)
diff


,ts_code,symbol,name,area,industry,list_date
0,301095.SZ,301095,广立微,浙江,半导体,20220805
1,688380.SH,688380,中微半导,深圳,半导体,20220805
2,688373.SH,688373,盟科药业,上海,化学制药,20220805
3,301308.SZ,301308,江波龙,深圳,IT设备,20220805
4,301278.SZ,301278,N快可,江苏,电气设备,20220804
5,001236.SZ,001236,弘业期货,江苏,多元金融,20220805
6,835985.BJ,835985,海泰新能,None,None,20220808


In [498]:
# 对比 tushare 上的股票列表与数据库的股票列表， 有差异时，补充差异的股票数据
def updata_stock_code_to_db():
    sc = obj_db.get_all_stock_code(table_name= table_name)
    scl=[]
    for i in range(0,len(sc)):
        scl.append(sc[i][0])

    sb2 = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')

    temp1 = sb2.loc[:,"ts_code"].values
    temp1 = list(temp1)

    temp2 = list(scl)

    ss =list(set(temp1) - set(temp2))

    if( len(ss) >0):
        diff = sb2[sb2.ts_code == ss[0]]
        for i in range(1,len(ss)):
            diff = pd.concat([diff, sb2[sb2.ts_code == ss[i]]], ignore_index=True)

    
    #store_stock_daily_to_mySQL(diff)


%%time
val_ret.drop('aa',axis = 1, inplace = True)
